## LDA Topic Modeling in Python
The tutorial can be found [here](https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html)

In [1]:
from nltk.corpus import twitter_samples, TwitterCorpusReader
import sys
sys.path.append("../bhtsa")
from process_twt import preprocess, get_stopwords, get_slang_dict

fileIds = twitter_samples.fileids()
root = twitter_samples.root
# read some negative tweet data from corpus
negReader = TwitterCorpusReader(root, fileIds[0])
negTwt = []
for tweet in negReader.docs():
    negTwt.append((tweet['text']))
# take a look at some of the tweets
for twt in negTwt[:10]:
    print twt

hopeless for tmr :(
Everything in the kids section of IKEA is so cute. Shame I'm nearly 19 in 2 months :(
@Hegelbon That heart sliding into the waste basket. :(
“@ketchBurning: I hate Japanese call him "bani" :( :(”

Me too
Dang starting next week I have "work" :(
oh god, my babies' faces :( https://t.co/9fcwGvaki0
@RileyMcDonough make me smile :((
@f0ggstar @stuartthull work neighbour on motors. Asked why and he said hates the updates on search :( http://t.co/XvmTUikWln
why?:("@tahuodyy: sialan:( https://t.co/Hv1i0xcrL2"
Athabasca glacier was there in #1948 :-( #athabasca #glacier #jasper #jaspernationalpark #alberta #explorealberta #… http://t.co/dZZdqmf7Cz


In [121]:
# preprocess them and show them again
import re
processed_twt = [preprocess(twt) for twt in negTwt]
p_twt = []
for twt in processed_twt:
    twt = re.sub('\'m', 'm', twt)
    twt = re.sub('\'t', 't', twt)
    twt = re.sub('\'s', '', twt)
    p_twt.append(twt)
for twt in p_twt[:10]:
    print twt

hopeless for tmr :(
everything in the kids section of ikea is so cute. shame im nearly 19 in 2 months :(
AT_USER that heart sliding into the waste basket. :(
“AT_USER i hate japanese call him "bani" :( :(” me too
dang starting next week i have "work" :(
oh god, my babies' faces :( URL
AT_USER make me smile :((
AT_USER AT_USER work neighbour on motors. asked why and he said hates the updates on search :( URL
why?:("AT_USER sialan:( URL
athabasca glacier was there in 1948 :-( athabasca glacier jasper jaspernationalpark alberta explorealberta … URL


In [122]:
# remove stop words and transfer slangs
import re
stop_words = get_stopwords()
slang_dict = get_slang_dict()
doc_set_removed = []
for twt in p_twt:
    removed = []
    words = twt.split()
    for w in words:
        # strip punctuation
        w = w.strip('\'"?,.')
        # check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        # ignore if it is a stop word
        if w.strip() in slang_dict:
            w = slang_dict[w.strip()]
        if w in stop_words or val is None:
            continue
        else:
            removed.append(w)
    doc_set_removed.append(removed)
print doc_set_removed[1]

[u'kids', u'section', u'ikea', u'cute', u'shame', 'Instant Message', u'nearly', u'months']


In [123]:
# tokenization
doc_set = []
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
for twt in doc_set_removed:
    tokens = doc_set.append(tokenizer.tokenize(' '.join(twt)))
print doc_set[1]

[u'kids', u'section', u'ikea', u'cute', u'shame', u'Instant', u'Message', u'nearly', u'months']


In [124]:
# stemming
from nltk.stem.porter import PorterStemmer
p_stemmer = PorterStemmer()
texts = []
for txt in doc_set:
    texts.append([p_stemmer.stem(i) for i in txt])
print texts[1]

[u'kid', u'section', u'ikea', u'cute', u'shame', u'Instant', u'Messag', u'nearli', u'month']


In [125]:
# construct document matrix
from gensim import corpora, models
dictionary = corpora.Dictionary(texts)
# convert to bag of words
corpus = [dictionary.doc2bow(text) for text in texts]
print corpus[0]

[(0, 1), (1, 1)]


In [128]:
# apply lda model, this might take a long time
import gensim
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=100)

In [127]:
ldamodel.print_topics(num_topics=5, num_words=10)

[(0,
  u'0.011*"peopl" + 0.010*"wish" + 0.010*"cant" + 0.009*"aw" + 0.009*"hate" + 0.008*"poor" + 0.008*"cute" + 0.008*"aww" + 0.008*"week" + 0.007*"ice"'),
 (1,
  u'0.052*"t" + 0.051*"don" + 0.033*"to" + 0.018*"want" + 0.011*"didnt" + 0.010*"go" + 0.008*"know" + 0.008*"watch" + 0.008*"I" + 0.008*"rain"'),
 (2,
  u'0.054*"miss" + 0.021*"oh" + 0.018*"time" + 0.018*"cant" + 0.017*"my" + 0.012*"laugh" + 0.010*"God" + 0.009*"out" + 0.009*"loud" + 0.009*"yeah"'),
 (3,
  u'0.053*"pleas" + 0.042*"follow" + 0.019*"love" + 0.019*"thank" + 0.015*"hope" + 0.014*"sorri" + 0.009*"okay" + 0.009*"get" + 0.008*"time" + 0.007*"applic"'),
 (4,
  u'0.075*"Messag" + 0.075*"Instant" + 0.019*"feel" + 0.014*"snapchat" + 0.012*"kik" + 0.012*"sad" + 0.011*"cant" + 0.009*"sorri" + 0.009*"tire" + 0.009*"look"')]